# OrthodoxAI - RAG

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

This notebook is dedicated to prototyping and testing **Retrieval-Augmented Generation (RAG)** methods within the **OrthodoxAI** framework.

It serves as a sandbox to:

- Evaluate different libraries and methods for building vector stores  
- Benchmark the performance and timing of the retrieval process  
- Isolate and debug components before integrating them into the main agent system

This notebook is focused solely on the retrieval layer of RAG, without involving agent roles or generation logic at this stage.

## Libraries

In [1]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

## Chroma Vector Store

### OpenAI

* Create the vector store

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma

In [3]:
JSON_DIR = "../src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/"

loader = DirectoryLoader(
    JSON_DIR,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

In [4]:
documents[-1]

Document(metadata={'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/3165_ΑΠΑΝΤΗΣΕΙΣ_ΑΠΟΡΙΩΝ_ΑΝΩΤ_ΚΑΤΗΧΗΤΙΚΟΥ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json', 'seq_num': 1}, page_content="Και πηγαίνουμε πιο κάτω. Ποια είναι η στάση των ανθρώπων μπροστά στο θάνατο; Τι θα μπορούσαμε να πούμε πάνω σε αυτό; Ξέρετε ότι σήμερα οι άνθρωποι στέκονται μπροστά στο θάνατο, να πω τη φράση του Αποστόλου Παύλου; Ως άθεοι εν κόσμω ελπίδα μη έχοντες. Ξέρετε τι σημαίνει αυτό; Αγνοούν την ανάσταση των νεκρών. Ως άθεοι εν κόσμω, ελπίδα μη έχοντες. Τι ελπίδα; Αναστάσεως των νεκρών. Τι κρίμα! Λέει ο Απόστολος Παύλος στους Θεσσαλονικείς , Δεν θέλω να λυπόσαστε όπως και οι λοιποί οι μη έχοντες ελπίδα, αυτοί που δεν έχουν ελπίδα αναστάσεως των νεκρών. Παιδιά, η στάση των ανθρώπων μπροστά στο θάνατο, των ποιών ανθρώπων; Των σωστών, των Χριστιανών, είναι ότι ο Χριστός ανεστήθη και εφόσον ανεστήθη ο Χριστός, θα αναστηθούν και οι νεκροί. Και ο Χριστός είναι ο πρωτότ

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [6]:
chunks[0].page_content

"Πρέπει να ευχαριστήσουμε τον Άγιο Τριαδικό Θεό μας, παιδιά , που μας αξίωσε πάλι να επανέλθουμε εις τον τόπον αυτόν, να ακουστεί ο Θείος του Λόγος. . Στην Αρχαία Εκκλησία η Βάπτισις εγίνεται εις μεγάλην ηλικίαν. Επειδή τώρα υπάρχει ο νηπιο-νηπιοβαπτισμός, η κατήχηση, όπως καταλαβαίνετε, γίνεται-γίνεται η κατήχηση, γίνεται μετά το βάπτισμά μας. Γι' αυτό το λόγο είναι μια αληθινή ευλογία αν κάποτε καταλάβουμε ότι πρέπει να συμπληρώσουμε την κατήχησή μας μετά το βάπτισμά μας, όταν μεγαλώσουμε. Δεν είναι ούτε πολυτέλεια η κατήχηση, ούτε κάτι το έκτακτο ή το, αν θέλετε, το προαιρετικό. Η κατήχηση είναι αναγκαιοτάτη. Γι' αυτό το λόγο, άμα συνειδητο-συνειδητοποιήσομε ότι πρέπει να αποκτήσωμε τη γνώση του περιεχομένου της πίστεώς μας, τότε αναμφισβήτητα θα τρέχουμε όχι στα παιδικά μας και εφηβικά μας χρόνια να συμπληρώνουμε τη γνώση μας, την πνευματική, τη θεολογική, τη χριστιανική, αλλά σε ολόκληρη τη ζωή μας. Οι μαθηταί του Χριστού, μέχρι που πέθαναν, ελέγοντο μαθηταί, που σημαίνει είχανε"

In [7]:
len(chunks)

8574

In [12]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", chunk_size=10) # nomic-embed-text-v1 is god 

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="athanasios-muthlinaios",
)
vectordb.persist()

/tmp/ipykernel_9623/4027366378.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [8]:
del vectordb

* Reload the vector store and query the data

In [3]:
# Re-load your persisted DB
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="athanasios-muthlinaios",
)

/tmp/ipykernel_7236/4170041564.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [4]:
query = "What did Athanasios Mytilenaios say about humidity?"
results = vectordb.similarity_search_with_score(query, k=5)

# Inspect docs and their distances
for i, (doc, distance) in enumerate(results):
    print(f"--- Result {i} ---")
    print(f"Similarity (higher is better): {distance:.4f}")
    print("Content:", doc.page_content.replace("\n", " "), "…")
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Similarity (higher is better): 1.3464
Content: κάποιους επισκέπτες και δεν είχε νερό. Χρειάστηκε μια κατσαρόλα νερό να βράσει λίγα όσπρια για να τους φυλάξει. Δεν είχε νερό. Τότε κάνει μια προσευχή και λέει. Κύριε του ουρανού και της γης, Συ που δίνεις των ηετών τη βροχή από τον ουρανό, δώσε μου λίγο νερό να βράσω τα όσπρια, για να φυλάξω τους επισκέπτες μου. Πριν τελειώσει την προσευχή του αμέσως ένα συννεφάκι από πάνω το συννεφάκι αυτό άρχισε να βρέχει μόνο στο σημείο που ήταν ο ασκητής και απέκτησε η κατσαρόλα του τόσο νερό όσο χρειάστηκε για να βράσει τα όσπρια. Κι όμως δεν έβρεξε σε όλη την περιοχή, παρά μόνο σε εκείνο το σημείο που χρειάστηκε να μαζέψει ο ασκητής λίγο νεράκι. Για το Θεό αγαπητοί μου, για το Θεό υπάρχει αδυναμία; Βγήκε λοιπόν άνεμος από τη θάλασσα δηλαδή από τον Ελαμιτικών κόλπον. Ήρθε ο άνεμος αυτός και παρέσυρε ορτιγομήτραν. Πουλιά. Είναι γνωστό ότι τα ορτύκια, ορτιγομήτρα λέγεται δηλαδή ορτυγομάνα θα το λέγαμε σήμερα στη γλώσσα μας, δηλαδή πλήθ

In [5]:
retriever = vectordb.as_retriever(search_kwargs={"k": 12})

In [6]:
query = "What did Athanasios Mytilenaios say about humidity?"
results = retriever.invoke(query)

# Inspect docs and their distances
for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print("Content:", doc.page_content.replace("\n", " "))
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Content: κάποιους επισκέπτες και δεν είχε νερό. Χρειάστηκε μια κατσαρόλα νερό να βράσει λίγα όσπρια για να τους φυλάξει. Δεν είχε νερό. Τότε κάνει μια προσευχή και λέει. Κύριε του ουρανού και της γης, Συ που δίνεις των ηετών τη βροχή από τον ουρανό, δώσε μου λίγο νερό να βράσω τα όσπρια, για να φυλάξω τους επισκέπτες μου. Πριν τελειώσει την προσευχή του αμέσως ένα συννεφάκι από πάνω το συννεφάκι αυτό άρχισε να βρέχει μόνο στο σημείο που ήταν ο ασκητής και απέκτησε η κατσαρόλα του τόσο νερό όσο χρειάστηκε για να βράσει τα όσπρια. Κι όμως δεν έβρεξε σε όλη την περιοχή, παρά μόνο σε εκείνο το σημείο που χρειάστηκε να μαζέψει ο ασκητής λίγο νεράκι. Για το Θεό αγαπητοί μου, για το Θεό υπάρχει αδυναμία; Βγήκε λοιπόν άνεμος από τη θάλασσα δηλαδή από τον Ελαμιτικών κόλπον. Ήρθε ο άνεμος αυτός και παρέσυρε ορτιγομήτραν. Πουλιά. Είναι γνωστό ότι τα ορτύκια, ορτιγομήτρα λέγεται δηλαδή ορτυγομάνα θα το λέγαμε σήμερα στη γλώσσα μας, δηλαδή πλήθος, πλήθος, πάρα πολύ πλήθος. Όχι απλώ

### Ollama Nomic

* Create the vector store

In [2]:
from langchain_community.document_loaders import JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from tqdm.auto import tqdm

In [3]:
# Load & split your transcripts
dir_path = "../src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/"
loader = DirectoryLoader(
    dir_path,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
print(f"Total chunks to embed: {len(chunks)}")

Total chunks to embed: 8574


In [4]:
# Initialize Chroma DB with Ollama embeddings
persist_dir = "vectorstore/chroma_db_ollama_nomic"
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://127.0.0.1:11434",
)
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_dir,
    collection_name="athanasios-muthlinaios",
)

# Embed & add each chunk with tqdm progress bar
for i, chunk in enumerate(tqdm(chunks, desc="Embedding chunks"), start=1):
    chunk_id = f"{chunk.metadata.get('source', 'doc')}_chunk_{i}"
    chunk.metadata["chunk_id"] = chunk_id
    vectordb.add_documents([chunk], ids=[chunk_id])

print(f"✅ All {len(chunks)} chunks embedded via nomic-embed-text-v1 and saved.")

Embedding chunks:   0%|          | 0/8574 [00:00<?, ?it/s]

✅ All 8574 chunks embedded via nomic-embed-text-v1 and saved.


In [14]:
del vectordb

* Reload the vectore store and query the data

In [15]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://127.0.0.1:11434",
)

vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="vectorstore/chroma_db_ollama_nomic",
    collection_name="athanasios-muthlinaios",
)

In [16]:
docs = vectordb.similarity_search(
    "What did Athanasios Mytilenaios say about humidity?",
    k=5
)
for i, doc in enumerate(docs, 1):
    print(f"--- Result {i} ---")
    print(doc.page_content.replace("\n", " "))
    print("► Source:", doc.metadata.get("source"))
    print("► Chunk ID:", doc.metadata.get("chunk_id"))
    print()

--- Result 1 ---
Ποτέ. Τι λέγει ο Ψαλμωδός; Κύριος Ποίμεν με και ουδέν με υστερήσει. Εις τόπον χλωρίς ευχή με κατεσκήνωσε. Κύριος είναι ο τσομπάνης μου, εγώ είμαι το κορβάδι του. Ο Κύριος με τιμαίνει και σε τίποτε δεν θα με υστερήσει. Τι έχω να φοβηθώ; Τι έχω να πάθω; Τίποτα. Γιατί; Γιατί ο Θεός είναι ο Πατέρας μου και ο Πατέρας ολοκλήρου της Δημιουργίας.
► Source: /mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json
► Chunk ID: /mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json_chunk_426

--- Result 2 ---
είναι ο αντίχριστος. Πότε; Όταν οι άνθρωποι δεν θα αναγνωρίζουν τον Αντίχριστο και θα πλανώνται, θα το θεωρούν ότι είναι μέγας φιλάνθρωπος. Θέλετε μια αποδείξει; Αγαπητοί μου, αυτή την εποχή τώρα ζούμε τους προδρόμους του αντιχρίστου και βλέπετε τους Χριστιανούς μας και δια

In [17]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
results = retriever.get_relevant_documents("Tell me about moisture in his speeches")
for i, doc in enumerate(results, 1):
    print(f"[{i}]", doc.page_content.replace("\n", " "), "…")
    print("   →", doc.metadata.get("source"), doc.metadata.get("chunk_id"))
    print()

[1] αυτή η περιέργεια. Είναι εκείνο που έλεγαν οι αρχαίοι Έλληνες, ότι «φύσει άνθρωπος ορέγετε του είδεναι». Εκ φύσεως ο άνθρωπος επιθυμεί να γνωρίζει. Δηλαδή αυτή η επιθυμία να γνωρίζει, η περιέργεια να γνωρίσει. Η περιέργεια ως τάχης γνωστική θα έκανε βέβαια τους πρωτοπλάστους να γνωρίσουν βαθύτερα και βαθύτερα τον δημιουργό τους, αλλά το είδος είναι φιλαργύρευμα.Και με τα πτωτικά ακόμα δεν θα ήταν παρά μία η περιέργεια, ή καλώς νοούμενοι, παρά μια αναζήτησις του αρχετύπου των, δηλαδή του ενανθρωπήσαντος Θεού Λόγου. Έτσι η περιέργεια με την καλή της, την αγαθή της διάσταση εμφανίζεται ως φιλομάθεια, σαν μια δίψα της ψυχής, μια δίψα του ανθρώπου για τη γνώση, για την επιστήμη. Λέγει θαυμάσια ο Ισοκράτης, στον Δημώνικο, του γράφει «Εάν εισ φιλομαθής, έστι και πολυμαθής». Εάν, λέγει, εε, είσαι φιλομαθής, αγαπάς να μαθαίνεις, τότε θα γίνεις και πολυμαθής. Θα γνωρίσεις πολλά πράγματα ακόμη «και αν μεν επίστασαι φίλασαι ταις μελέταις, αν δε μη ναι μάθηκας προσλάμβανεται τις επιστήμινες», …

### Ollama MXBAI

* Create the vector store

In [2]:
from langchain_community.document_loaders import JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from tqdm.auto import tqdm

In [3]:
# Load & split your transcripts
dir_path = "../src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/"
loader = DirectoryLoader(
    dir_path,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
print(f"Total chunks to embed: {len(chunks)}")

Total chunks to embed: 8574


In [5]:
# Initialize Chroma DB with Ollama embeddings
persist_dir = "vectorstore/chroma_db_ollama_mxbai"
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://127.0.0.1:11434",
)
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_dir,
    collection_name="athanasios-muthlinaios",
)

# Embed & add each chunk with tqdm progress bar
for i, chunk in enumerate(tqdm(chunks, desc="Embedding chunks"), start=1):
    chunk_id = f"{chunk.metadata.get('source', 'doc')}_chunk_{i}"
    chunk.metadata["chunk_id"] = chunk_id
    vectordb.add_documents([chunk], ids=[chunk_id])

print(f"✅ All {len(chunks)} chunks embedded via nomic-embed-text-v1 and saved.")

Embedding chunks:   0%|          | 0/8574 [00:00<?, ?it/s]

✅ All 8574 chunks embedded via nomic-embed-text-v1 and saved.


In [6]:
del vectordb

* Reload the vectore store and query the data

In [10]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://127.0.0.1:11434",
)

vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="vectorstore/chroma_db_ollama_mxbai",
    collection_name="athanasios-muthlinaios",
)

In [11]:
docs = vectordb.similarity_search(
    "What did Athanasios Mytilenaios say about humidity?",
    k=5
)
for i, doc in enumerate(docs, 1):
    print(f"--- Result {i} ---")
    print(doc.page_content.replace("\n", " "))
    print("► Source:", doc.metadata.get("source"))
    print("► Chunk ID:", doc.metadata.get("chunk_id"))
    print()

--- Result 1 ---
ο γιος της οικογενείας που έρχεται η κόρη τώρα μέσα στο σπίτι και ομοίως να γίνει-- να γίνεται κατάθεσις των χρημάτων εις τους γονείς και θα πρέπει τώρα η νύφη, ακόμα χειρότερο αυτό, να πάει να πει στον πεθερό πατέρα, ξέρεις, θέλω μια φούστα, θέλω ένα φουστάνι.Και ο πατέρας να είναι εκείνος που θα κρίνει αν θα πρέπει να αγοράσει η νύφη καινούργιο φουστάνι ή όχι και όχι ο σύζυγος. Αυτό συμβαίνει σε πολλά σπίτια. Αγαπητοί μου. Ερωτώ πάλι Μπορεί να αισθάνεται μια τέτοια κοπέλα ένας τέτοιος νέος μέσα στο σπίτι, σε ένα τέτοιο σπίτι άνετα και να ωριμάσουν σαν πρόσωπα; Ποτέ. Αν υποτεθεί ότι ή εσείς θα καλέσετε ένα παιδί να γίνει στο σπίτι σας γαμπρός ή δεν ξέρω πώς, πάντως εγώ να πω τούτο. Να μάθουμε αγαπητοί μου το εξής Μπορούμε να βοηθήσουμε τα παιδιά μας, αλλά γρήγορα θα αποσυρθούν και θα κυβερνήσουν τα παιδιά μας μέσα στο σπίτι. Εάν φοβείστε ότι δεν θα δύναστε να κρατάτε εσείς εκείνα που θέλετε να κρατάτε, τουλάχιστον μερικά οικονομικά, θα σας έλεγα Κρατήστε ένα μέρος της

In [13]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
results = retriever.get_relevant_documents("Tell me about moisture in his speeches")
for i, doc in enumerate(results, 1):
    print(f"[{i}]", doc.page_content.replace("\n", " "), "…")
    print("   →", doc.metadata.get("source"), doc.metadata.get("chunk_id"))
    print()

[1] ο Κύριος, ο Θεός του Παραδείσου. Για σκεφτείτε αυτό το πράγμα. Σκεφτείτε τι μεγάλη τιμή έχει ο άνθρωπος. Αυτά που τα λέμε τόσο απλά. Ξέρετε τι μεγάλη αξία έχουν; Ξέρετε ότι σήμερα οι άνθρωποι προσπαθούν να δώσουν, να κατανοήσουν με τη φιλοσοφία τι είναι ο άνθρωπος; Και όταν κάποτε ανακαλύπτουν την αξία άνθρωπος διακηρύσσουν, ξέρετε, λέει, ο άνθρωπος είναι πρόσωπον. Χαίρω πολύ, θα τους λέγαμε. Είναι πρόσωπον. Τώρα το ανακαλύψατε. Ναι, λέει, δεν είναι μάζα. Είναι πρόσωπον ο άνθρωπος. Μα αυτό το ξέρουμε εμείς από την αποκάλυψη του Θεού. Όχι μόνο ότι ο άνθρωπος δεν είναι πρόσωπον, αλλά είναι ένας μικρός θεός επάνω στη γη, με προορισμό να θεωρηθεί. Και όταν αργότερα ο διάβολος θα του πει του Αδάμ Σας είπε ο Θεός να μη δοκιμάσετε από τον καρπόν για να μη γίνετε θεοί, εδώ έλεγε ένα ψέμα ο διάβολος αλλά και μιαν αλήθεια. Η αλήθεια είναι ότι πραγματικά θα εγίνω το θεοί. Όχι όμως τρώγοντας τον καρπό του δέντρου που ο Θεός είπε να μη δοκιμάσω, αλλά θα εγίνουν το θεοί έχοντας την εξάρτησή τους

### Hybrid Search

In [ ]:
from langchain.retrievers import HybridSearchRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever  # e.g. from langchain.retrievers import BM25Retriever

# 1) your vector-store retriever
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="athanasios-muthilinaios",
)
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# 2) a simple BM25 / keyword retriever over the same docs 
#    (you can point it at the same text source you used to build Chroma)
loader   = DirectoryLoader(JSON_DIR, glob="*.json", loader_cls=JSONLoader, loader_kwargs={"jq_schema": ".content"})
docs     = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks   = splitter.split_documents(docs)
bm25     = BM25Retriever.from_documents(chunks)    # which gives you a .get_relevant_documents()

# 3) glue them together
hybrid = HybridSearchRetriever.from_retrievers([
    vector_retriever, 
    bm25
], weights=[0.7, 0.3])  # e.g. 70% vector score + 30% BM25 score

# 4) run your query
query   = "What did Athanasios Mytilenaios say about humidity?"
results = hybrid.get_relevant_documents(query)

for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print(doc.page_content)
    print(doc.metadata)
    print()


## Qdrant Vector Store